<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [8]</a>'.</span>

## Load data

In [1]:
import pandas as pd

In [2]:
df_base = pd.read_csv('../data/general/pjm_pivot.csv', index_col=0, parse_dates=True)
df_base.head()

,AE,AEP,AP,ATSI,BC,CE,DAY,DEOK,DOM,DPL,DUQ,EKPC,JC,ME,PE,PEP,PL,PN,PS,RECO
period,,,,,,,,,,,,,,,,,,,,
2018-07-01 05:00:00,1301,12187,4384,6960,3134,12136,1619,2895,9775,1919,1466,1209,2388,1442,4397,3165,3835,1611,5009,186
2018-07-01 06:00:00,1314,11946,4391,6762,3139,11872,1605,2857,9787,1950,1455,1197,2416,1465,4423,3156,3901,1641,4990,187
2018-07-01 07:00:00,1410,12664,4757,6670,3377,11992,1707,2997,10453,2160,1528,1273,2644,1605,4743,3332,4232,1728,5267,202
2018-07-01 08:00:00,1567,14069,5308,7065,3788,12860,1916,3331,11734,2470,1688,1472,3064,1784,5230,3679,4613,1899,5735,230
2018-07-01 09:00:00,1749,15610,5862,7833,4262,14212,2145,3703,13084,2765,1875,1656,3569,1972,5752,4085,5014,2055,6299,259


## Select region

In [3]:
region = 'AE'

In [4]:
# Parameters
region = "RECO"


In [5]:
df = df_base.loc[:, [region]]
df

,RECO
period,
2018-07-01 05:00:00,186
2018-07-01 06:00:00,187
...,...
2023-11-01 03:00:00,141
2023-11-01 04:00:00,131


## Data Preprocessing

### Train Test Split

In [6]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2, shuffle=False)

### Export Data

In [7]:
import os

path_region = f'../data/regions/{region}'

if not os.path.exists(path_region):
    os.makedirs(path_region)
    
train.to_csv(f'{path_region}/train.csv')
test.to_csv(f'{path_region}/test.csv')

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [8]:
x

NameError: name 'x' is not defined

### Scale Data

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0, 1))

train_norm = scaler.fit_transform(train)
test_norm = scaler.transform(test)

### Create Sequences

In [ ]:
import numpy as np

def create_sequences(data, sequence_length):
    X, y = [], []
    for i in range(len(data) - sequence_length):
        X.append(data[i:i+sequence_length])
        y.append(data[i+sequence_length])
    return np.array(X), np.array(y)

sequence_length = 24 # Use 24 hours prior to predict the following hour
X_train, y_train = create_sequences(train, sequence_length)
X_test, y_test = create_sequences(test, sequence_length)

## Modelling

### Design NN Architecture

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense

model = Sequential()
model.add(LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')

### Train Model

In [ ]:
from keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', patience=5)

history = model.fit(
    X_train, y_train, epochs=50, batch_size=64, verbose=0,
    validation_data=(X_test, y_test), callbacks=[early_stop])

### Calculate Predictions

In [ ]:
y_pred = model.predict(X_test)
y_pred

### Comparison: Real Data & Prediction

#### Descale Data

In [ ]:
y_pred_scaled_inverse = scaler.inverse_transform(y_pred)
y_true = scaler.inverse_transform(y_test)

#### Create DataFrame

In [ ]:
dic_pred = {
    'y_pred': y_pred.flatten(),
    'y_pred_scaled_inverse': y_pred_scaled_inverse.flatten(),
    'y_true': y_true.flatten(),
}

df_pred = pd.DataFrame(dic_pred)
df_pred

#### Evaluate Model

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
mean_squared_error(df_pred.y_true, df_pred.y_pred_scaled_inverse, squared=False)

In [ ]:
model.save(f'../models/{region}.keras')